In [1]:
!pip install transformers
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Kagsot/

Mounted at /content/drive/
/content/drive/MyDrive/Kagsot


In [ ]:
t1=pd.read_csv('train.csv')
t2=pd.read_csv('test.csv')
t3=pd.read_csv('submit.csv')
t2['label']=t3['label']


FileNotFoundError: ignored

In [ ]:
t=pd.concat([t2,t1])

In [ ]:
fa=pd.read_csv('Fake.csv')
tr=pd.read_csv('True.csv')

In [ ]:
fa['label']=0
tr['label']=1
t=pd.concat([fa,tr])

In [3]:
trb=pd.read_csv('trainbasekagsot.csv')
tbtm=pd.read_csv('test_base_&_train_metakagsot.csv')
tm = pd.read_csv('test_metakagsot.csv')


In [ ]:
tr=pd.read_csv('subsetforbagkagsot.csv')
tst=pd.read_csv('testforbagkagsot.csv')
# r=pd.read_csv('realnews_collected_by_us.csv')
# r['label']=r['class']
# r['text']=r['title_text']
# r=r.drop('class', axis=1)
# r=r.drop('title_text',axis=1)
# trst=pd.concat([tst,r])

In [ ]:
tr=tr.dropna()
tr['label']=tr['label'].astype(int)
tst=tst.dropna()
tst['label']=tst['label'].astype(int)

<ipython-input-4-695a09b4cc9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr['label']=tr['label'].astype(int)


In [ ]:
sz=len(tr)
def modifydatabag(t1):
  res=[]
  a=int(sz*0.90)
  start=0
  end=len(t1)
  for i in range(a):
    res.append(t1.iloc[random.randrange(start,end)])
  return res
#pass only test dataset for modification
data=modifydatabag(tr)
df = pd.DataFrame(data, columns=['text','label'])
df.head(1)

,text,label
5073,Looming national security laws raise fresh fea...,1


In [4]:
# trb.head(2)
# tbtm.head(2)
trb=trb.dropna()
tbtm=tbtm.dropna()
tm=tm.dropna()
trb['label']=trb['label'].astype(int)
tbtm['label']=tbtm['label'].astype(int)
tm['label']=tm['label'].astype(int)


<ipython-input-4-131d9d675076>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trb['label']=trb['label'].astype(int)
<ipython-input-4-131d9d675076>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbtm['label']=tbtm['label'].astype(int)


In [ ]:

print(tm['text'].isna())

0        False
1        False
2        False
3        False
4        False
         ...  
10098    False
10099    False
10100    False
10101    False
10102    False
Name: text, Length: 10103, dtype: bool


In [5]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )




In [6]:
token_id = []
attention_masks = []
labels=trb.label.values
tex=trb.text.values
for sample in tex:
  encoding_dict = preprocessing(str(sample), tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(type(trb.label.values))
print(type(r.label.values))
r=pd.read_csv('realnews.csv')

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
r.dropna()

,Unnamed: 0,label,text
0,0,1,photoshopped image of swami vivekananda playin...
1,1,1,drug kingpin joaquin el chapo guzman escapes m...
2,2,1,europe is not experiencing medieval conditions...
3,3,0,four proud boys convicted of sedition in key j...
4,4,0,chris christie taunts trump as afraid of presi...
...,...,...,...
196,197,0,with default looming what can biden do the fed...
197,198,1,the false claim of a faked war debunking misin...
198,199,0,biden and age questions about biden and age wi...
199,200,0,big promises from vivek ramaswamy vivek ramasw...


In [18]:
token_id2 = []
attention_masks2 = []
labels2=tm.label.values
tex2=tm.text.values
for sample in tex2:
  encoding_dict = preprocessing(str(sample), tokenizer)
  token_id2.append(encoding_dict['input_ids']) 
  attention_masks2.append(encoding_dict['attention_mask'])


token_id2 = torch.cat(token_id2, dim = 0)
attention_masks2 = torch.cat(attention_masks2, dim = 0)
labels2 = torch.tensor(labels2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle=True,
    stratify=labels
    )
# Train and validation sets
train_set = TensorDataset(token_id, 
                          attention_masks, 
                          labels)

val_set_b = TensorDataset(token_id2, 
                        attention_masks2, 
                        labels2)

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set_b,
            sampler = SequentialSampler(val_set_b),
            batch_size = batch_size
        )

In [9]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [10]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )
# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        
        #print(model)
        # Forward pass
        # print(b_input_ids.shape)
        # print(b_input_mask.shape)
        # print(b_labels.shape)
        train_output = model(
            b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask,
                             labels = b_labels
                             )
        #Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

   

Epoch: 100%|██████████| 2/2 [28:55<00:00, 867.80s/it]


In [32]:
 # Set model to evaluation mode
model.eval()
val_labels=[]
val_cal=[]
# Tracking variables 
val_accuracy = []
val_precision = []
val_recall = []
val_specificity = []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      # Forward pass
      eval_output = model(b_input_ids, 
                          token_type_ids = None, 
                          attention_mask = b_input_mask)
    logits = eval_output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # Calculate validation metrics
    b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
    val_labels.append(label_ids)
    val_cal.append(logits)
    val_accuracy.append(b_accuracy)
    # Update precision only when (tp + fp) !=0; ignore nan
    if b_precision != 'nan': val_precision.append(b_precision)
    # Update recall only when (tp + fn) !=0; ignore nan
    if b_recall != 'nan': val_recall.append(b_recall)
    # Update specificity only when (tn + fp) !=0; ignore nan
    if b_specificity != 'nan': val_specificity.append(b_specificity)

print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')



	 - Train loss: 0.0883
	 - Validation Accuracy: 0.9657
	 - Validation Precision: 0.9672
	 - Validation Recall: 0.9611
	 - Validation Specificity: 0.9697



In [33]:
# print(len(val_labels))
# print(type(val_labels))
# print(len(val_cal))
# print(type(val_cal))
x=[]
y=[]
for i in val_labels:
  for num in i:
    x.append(num)

for logits in val_cal:
  y_pred = np.argmax(logits, axis=1)
  for num in y_pred:
    y.append(num)

resdf = pd.DataFrame(y, columns=['label'])
resdf.head(3)
# df.to_csv('bertisotvot.csv', index=False)

,label
0,1
1,1
2,1


In [34]:
v=pd.read_csv('testmetakagsot.csv')

In [36]:
v['bert']=resdf['label']

print(v)

       svm  nb  lg  cnn  lstm  bert
0        0   0   0    0     0     1
1        1   0   1    1     1     1
2        0   0   0    0     0     1
3        1   1   1    1     1     1
4        1   1   1    1     1     1
...    ...  ..  ..  ...   ...   ...
14331    0   0   0    0     0     1
14332    1   0   1    1     1     1
14333    1   1   1    1     1     1
14334    0   0   0    0     0     0
14335    0   0   0    1     0     0

[14336 rows x 6 columns]


In [37]:
from sklearn.metrics import accuracy_score
print(accuracy_score(v['svm'], v['lstm']))

0.9312220982142857


In [38]:
v.to_csv('testmetakagsot.csv',index=False)